# c04-boats

*Purpose*: TODO


In [ ]:
import grama as gr
import pandas as pd
import time
DF = gr.Intention()
%matplotlib inline

# Boat analysis code
import boat_utils as boat


# Context


## Basics of Boat Hull Design

### TODO Boat schematic


### Statical Stability

(TODO exposition)

In [ ]:
# NOTE: No need to edit; this will analyze an example boat
t0 = time.time()
v_design = [2.5, 3.0, 2.0, 0.30, 0.35]
df_example = boat.fun_moment(v_design)
t1 = time.time()
print("Execution time: {0:4.3f} sec".format(t1 - t0))

(
    # Visualize the moment curve
    df_example
    >> gr.ggplot(gr.aes("angle * 180/3.14", "M_net"))
    + gr.geom_hline(yintercept=0, linetype="dashed")
    + gr.geom_line()
    
    + gr.scale_color_discrete(name=None)
    + gr.theme_minimal()
    + gr.labs(
        x="Boat Angle (Degrees)",
        y="Net Moment (lbf * in)",
    )
)

Some things worth noting about this example:

- The Boat Angle is the angle to which the boat hull is rotated (along its long axis).
- When the net moment is negative, a restoring torque acts to turn the boat back to upright.
- When the net moment is positive, an anti-restoring torque is applied.
<!-- - The point where the moment curve passes through the axis `M_net == 0` with positive slope is called the *angle of vanishing stability* (AVS). Rotating the boat beyond this point will cause it to settle upside-down. -->
- The boat analysis takes a non-trivial amount of time to run! Analyzing a large number of boat designs will take a fair amount of simulation time.


Note that upright stability is *not* guaranteed, even for a boat of identical geometry. The following boat has identical hull geometry but is lighter (smaller `d`) and has a higher center of mass (larger `f_com`).


In [ ]:
# NOTE: No need to edit; this will analyze an example boat
t0 = time.time()
v_light = [2.5, 3.0, 2.0, 0.20, 0.40]
df_light = boat.fun_moment(v_light)
t1 = time.time()
print("Execution time: {0:4.3f} sec".format(t1 - t0))

(
    # Visualize the moment curve
    df_light
    >> gr.tf_mutate(source="Light loading")
    >> gr.tf_bind_rows(
        df_example
        >> gr.tf_mutate(source="Baseline")
    )
    >> gr.ggplot(gr.aes("angle * 180/3.14", "M_net", color="source"))
    + gr.geom_hline(yintercept=0, linetype="dashed")
    + gr.geom_line()
    
    + gr.scale_color_discrete(name=None)
    + gr.theme_minimal()
    + gr.labs(
        x="Boat Angle (Degrees)",
        y="Net Moment (lbf * in)",
    )
)

Note that the loading conditions of the boat have destabilized it at upright; the slope of the moment curve at $0^{\circ}$ is now positive.

This phenomenon is dangerous for vessels that must operate in both heavily-loaded and lightly-loaded conditions: Imagine a large transport ship that has just unloaded cargo and is in-harbor in an unloaded configuration. The destabilizing phenomenon we see here would put the crew and dockworkers in considerable danger.


## Archival Data


In [ ]:
filename_archival = "./data/doe-wide.csv"
df_archival = pd.read_csv(filename_archival)
df_archival.head()

### Data Dictionary

| Column | I/O | Meaning |
|---|---|---|
| `d` | Input | Displacement ratio (-) |
| `n` | Input | Shape factor (-) |
| `f_com` | Input | Height of COM from boat bottom (-) |
| `H` | Input | Height of boat (in) |
| `W` | Input | Width of boat (in) |
|---|---|---|
| `M_max` | Output | Max righting moment (`lbf*in`) |
| `M_min` | Output | Min righting moment (`lbf*in`) |
| `mass` | Output | Boat mass (lb) |
| `dMdtheta_avs` | Output | Slope of moment curve at AVS (`lbf in/rad`) |
| `dMdtheta_0` | Output | Slope of moment curve at upright (`lbf*in/rad`) |
| `int_M_stable` | Output | Integral of moment curve, up to the AVS (`lbf*in*rad`) |
| `BM` | Output | Metacentric radius (in) |
| `GM` | Output | Metacentric height (in) |
| `angle` | Output | Angle of vanishing stability (rad) |


In [ ]:
var = ["d", "n", "f_com", "H", "W"]
out = [
    "M_max", 
    "M_min", 
    "mass", 
    "dMdtheta_avs", 
    "dMdtheta_0", 
    "BM", 
    "GM",
    "angle",
    "int_M_stable",
]

# EDA of Archival Data


### __qX__ First look

Use simple EDA techniques to answer the questions under *observations* below.


In [ ]:
# TASK: Perform simple EDA, answer the questions below
(
    df_archival

)

*Observations*

- How many observations (distinct boat hulls) are there in-total?
  - (Your response here)
- What are rough bounds for each of the input variables? (You'll use these in a later task....)
  - (Your response here)


### __qX__ How many designs are upright-stable?

Remember that "upright-stable" means the slope of the moment curve at an upright position is negative.


In [ ]:
(
    df_archival

)

*Observations*

- How many boat designs in the dataset are upright-stable?
  - (Your response here)
- How *fraction* of boat designs in the dataset are upright-stable?
  - (Your response here)


### __qX__ Correlation tile plot


In [ ]:
(
    df_archival
    ## Compute the input/output correlation data here

    ## NOTE: No need to edit; this will visualize your data
    >> gr.pt_auto()
)

*Observations*

- Which inputs correlate strongly with the slope of the moment curve at upright (`dMdtheta_0`)?
  - (Your response here)
- For each of the inputs you identified above, what is the sign of the correlation? (Positive or negative?) For each input, what kind of change (increase or decrease) is required to increase upright stability?
  - (Your response here)


### __qX__ Detailed scatterplots

Make scatterplots to study relationship between `dMdtheta_0` and the relevant inputs you identified above. Add a smooth trend to each scatterplot with `gr.geom_smooth()`.

*Hint*: While it is possible to construct all three plots in a single code cell, you should feel free to create additional cells as-necessary.


In [ ]:
# TODO: Create scatterplots


*Observations*

- Is the observed variability real or erroneous?
  - (Your response here)
- Describe the effects of each input on the output `dMdtheta_0`.
  - (Your response here)
- For each input, describe the variability around the `gr.geom_smooth()` trend: Is it small or large at different values of the input?) 
  - (Your response here)
- Do any of the inputs have a more *predictable* effect on `dMdtheta_0`?
  - (Your response here)


# Model Setup

```python
df_res = boat.fun_performance(v)
```


### __qX__ Time the analysis


In [ ]:
n_rep = 5

t0 = time.time()
for i in range(n_rep):

t1 = time.time()

t_exec = (t1 - t0) / n_rep

print("Average execution time: {0:} sec".format(t_exec))


### __qX__ Define the input order


In [ ]:
out_performance = [
    "mass",
    "GM",
    "angle",
    "dMdtheta_0",
    "dMdtheta_avs",
    "M_max",
    "M_min",
    "int_M_stable",
]



## NOTE: No need to edit; use this to check your work
assert \
    set(var_performance) == set(boat.var_correct), \
    "Inputs not correct"
assert \
    var_performance == boat.var_correct, \
    "Inputs are not in the correct order; check " + \
    "the documentation for boat.fun_performance()."

```{admonition} Why does order matter?
By default, Python functions interpret their inputs based on **order**. Grama takes care of variables based on **name**, but when working with a "regular" Python function, we have to be careful to give grama the correct order information. Imagine giving the value for boat height `H` as the shape factor `n`, or vice versa! If we are careful when defining a grama model, we can prevent these order issues later.
```

### __qX__ Assemble the model


In [ ]:
md_performance = (
    gr.Model("Boat performance")
    >> gr.cp_function(
        fun=boat.fun_performance,
        # Add input and output names;
        # runtime estimate

    )
    >> gr.cp_bounds(
        # Add input bounds

    )
)

## NOTE: No need to edit; use this to check your work
assert \
    md_performance.functions[0].runtime > 0, \
    "Model has no runtime estimate"
for v in var_performance:
    assert \
        md_performance.domain.bounds[v][0] < df_archival[v].min(), \
        "Archival data for {} exceeds model lower bound".format(v)
    assert \
        df_archival[v].max() < md_performance.domain.bounds[v][1], \
        "Archival data for {} exceeds model upper bound".format(v)

md_performance